In [43]:
import numpy as np
import matplotlib.pyplot as plt
import bisect
import time
import array


def fn_lieb_find_circle(lieb_up,lieb_down,index0,index1):
    (ind_out, isup, v_circle1) = fn_lieb_combine(lieb_up,lieb_down,index0,index1,True)
    
    # case 1
    if ind_out==index1 and isup==True:
        circle_number_change = 1
        if v_circle1:
            res = fn_lieb_combine(lieb_up,lieb_down,index0,index1,False)
            v_circle2 = res[2]
            if v_circle2:
                if len(v_circle1)>=len(v_circle2):
                    return circle_number_change, v_circle2, v_circle1
                else:
                    return circle_number_change, v_circle1, v_circle2
            else:
                return circle_number_change, [], []
        else:
            return circle_number_change, [], []
        
    # case 2
    elif ind_out==index1 and isup==False:
        circle_number_change = 0
        return circle_number_change, [], []
    
    # case 3
    elif ind_out==index0 and isup==False:
        circle_number_change = -1
        if v_circle1:
            res = fn_lieb_combine(lieb_up,lieb_down,index1,index0,True)
            v_circle2 = res[2]
            if v_circle2:
                if len(v_circle1)>=len(v_circle2):
                    return circle_number_change, v_circle2, v_circle1
                else:
                    return circle_number_change, v_circle1, v_circle2
            else:
                return circle_number_change, [], []
    else:
        print('error')
        
        
        
def fn_lieb_combine(lieb_up, lieb_down, index0, index1, isup0):
        N = len(lieb_up)//2      
        v_mid = []
        if isup0:
            ind = index0 + N
        else:
            ind = index0
        isup = isup0
        while True:
            if isup:
                ind = lieb_up[ind]
                if ind<=N-1:       
                    v_mid.append(ind)
                    ind = ind + N
                else:
                    ind = ind - N
                    if ind==index0 or ind==index1:
                        return ind, isup, v_mid
                isup = not isup
                
            else:
                ind = lieb_down[ind]
                if ind>=N:
                    ind = ind - N
                    v_mid.append(ind)
                else:
                    if ind==index0 or ind==index1:
                        return ind, isup, v_mid
                    ind = ind + N
                isup = not isup
            # print(ind)
    
    
def fn_lieb_identity(N):
    return [i+N for i in range(N)] + [i for i in range(N)]


def fn_lieb_add(lieb, p1, p2, v_index, v_value):
    t1 = lieb[p1]
    t2 = lieb[p2]
    if not t1 == p2:
        lieb[t1] = t2
        lieb[t2] = t1
        lieb[p1] = p2
        lieb[p2] = p1
        # v_index.append([t1,p1,t2,p2])
        # v_value.append([p1,t1,p2,t2])
        v_index.append(t1)
        v_index.append(p1)
        v_index.append(t2)
        v_index.append(p2)
        v_value.append(p1)
        v_value.append(t1)
        v_value.append(p2)
        v_value.append(t2)


class SN:
    def __init__(self,N,M):
        self.N = N
        self.M = M
        self.lmap = [fn_lieb_identity(N) for _ in range(M+1)]
        self.cn = N
        self.circle_index = [i for i in range(N)]
        self.stack_circle = [[], N]
        self.Hamiltonian = [[0 for _ in range(N)] for _ in range(N)]
        
        self.cn_backup = N
        self.v_for_stack = []
        self.v_modified_map_index = []
        self.v_modified_map_value = []
        self.modified_map_num = 0
        self.v_modified_circle_index = []
        self.v_modified_circle_value = []
        self.modified_circle_num = 0
        
        self.v_circle1 = []
        self.v_circle2 = []
      
    
    def fn_initialization(self):
        for layer in range(self.M-1, -1, -1):
            map_up = self.lmap[layer]
            map_down = self.lmap[layer + 1]
            leg_number = np.random.poisson(1)
            vp = [np.random.randint(1,N) for _ in range(leg_number)]
               
        
    def fn_stack_circle_pull(self):
        if not self.stack_circle[0]:
            num_circle = self.stack_circle[1]
            self.stack_circle[1] = num_circle + 1
            self.circle.append([])
        else:
            num_circle = self.stack_circle[0][-1]
            self.stack_circle[0].pop()
        return num_circle
    
    
    def fn_stack_circle_push(self,index):
        self.stack_circle[0].append(index)
        
        
    def fn_try_layer(self,layer):
        map_up = sn.lmap[layer]
        map_down = sn.lmap[layer+1]
        circle_number = sn.cn
        
        
    def fn_update_layer_forward(self, layer, legs):
        self.cn_backup = self.cn
        self.v_for_stack = []
        self.v_modified_map_index = []
        self.v_modified_map_value = []
        self.v_modified_circle_index = []
        self.v_modified_circle_value = []
        
        map_up = self.lmap[layer]
        map_down = self.lmap[layer+1]
        
        for leg in legs:
            p1 = leg[0]
            p2 = leg[1]
            (cn_change, v_circle1, v_circle2) = fn_lieb_find_circle(map_up,map_down,p1,p2)
            
            # change cn
            self.cn = self.cn + cn_change
            
            # change v_circle_index
            if v_circle1:
                if cn_change == 1:
                    circle_index = self.fn_stack_circle_pull()
                    self.v_for_stack.insert(0,circle_index)
                    for change_index in v_circle1:
                        self.v_modified_circle_index.append(change_index)
                        self.v_modified_circle_value.append(self.circle_index[change_index])
                        self.circle_index[change_index] = circle_index
                elif cn_change == -1:
                    self.fn_stack_circle_push(self.circle_index[v_circle1[0]])
                    circle_index = self.circle_index[v_circle2[0]]
                    self.v_for_stack.insert(0,-1)
                    for change_index in v_circle1:
                        self.v_modified_circle_index.append(change_index)
                        self.v_modified_circle_value.append(self.circle_index[change_index])
                        self.circle_index[change_index] = circle_index
                else:
                    print('error')
            
            # change map_up
            fn_lieb_add(map_up, p1, p2, self.v_modified_map_index, self.v_modified_map_value)        
                        
            

In [44]:
N = 4
M = 6
sn = SN(N,M)
legs = [[0,1],[0,2]]
sn.fn_update_layer_forward(1,legs)

In [51]:
print(sn.v_modified_circle_index)
print(sn.v_modified_circle_value)
print(sn.v_modified_map_index)
print(sn.v_modified_map_value)
print(sn.lmap[1])
print(sn.v_for_stack)

[1, 2]
[1, 2]
[4, 0, 5, 1, 1, 0, 6, 2]
[0, 4, 1, 5, 0, 1, 2, 6]
[2, 6, 0, 7, 5, 4, 1, 3]
[-1, -1]


In [33]:
fn_try(y)
y

hehe


[1, 2, 3, 1]

In [19]:
y = [[1,2],[3,4]]

In [20]:
for k in y:
    print(k)

[1, 2]
[3, 4]


In [21]:
y

[[1, 2], [3, 4]]

In [24]:
y.insert(0,1)

In [25]:
y

[1, 1, [1, 2], [3, 4]]